In [1]:
versioninfo()

Julia Version 1.4.0
Commit b8e9a9ecc6 (2020-03-21 16:36 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin18.6.0)
  CPU: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

#### Question 1: Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$.

I have a linear combination of normals, so the result will be normal. Then all I need is to find the mean and variance
$$
\begin{align*}
\mathbb EY_i|\beta,\Sigma,\sigma^2&=\mathbb E[X_i\beta+Z_i\gamma+\varepsilon_i]\\
&=X_i\beta\\
Var(Y_i|\beta,\Sigma,\sigma^2)&=Var(X_i\beta+Z_i\gamma+\varepsilon_i)\\
&=Z_i\Sigma Z_i^\top+\sigma^2\mathbf I.
\end{align*}
$$
Thus $Y_i|\beta,\Sigma,\sigma^2\sim N(X_i\beta, Z_i\Sigma Z_i^\top+\sigma^2\mathbf I)$.

Let $\Omega:=Z_i\Sigma Z_i^\top+\sigma^2\mathbf I$ so the log-likelihood can be written as
$$
-\frac{1}{2}\log|2\pi\Omega|-\frac{1}{2}(y_i-X_i\beta)^\top\Omega^{-1}(y_i-X_i\beta)
$$

In [301]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate arrays you may want to pre-allocate
    res        :: Vector{T}
    storage_q  :: Vector{T}
    storage_q2  :: Vector{T}
    ztz        :: Matrix{T}
    storage_qq :: Matrix{T}
    storage_qq2 :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}) where T <: AbstractFloat
    res        = similar(y)
    storage_q  = Vector{T}(undef, size(Z, 2))
    storage_q2  = Vector{T}(undef, size(Z, 2))
    ztz        = transpose(Z) * Z
    storage_qq = similar(ztz)
    storage_qq2 = similar(ztz)
    LmmObs(y, X, Z, res, storage_q, storage_q2, ztz, storage_qq, storage_qq2)
end

LmmObs

Computing the log-likelihood will involve some hairy terms. To start, I'll look at the quadratic form

$$
(y_i-X_i\beta)^\top\Omega^{-1}(y_i-X_i\beta)=(y_i-X_i\beta)^\top(\sigma^2 I+Z\Sigma Z^\top)^{-1}(y_i-X_i\beta)
$$

I'm given the Cholesky decomposition of $\Sigma$ as $LL^\top$. I'll rewrite $Z\Sigma Z^\top=ZLL^\top Z^\top$ as $\tilde Z\tilde Z^\top$, with $\tilde Z:=ZL$. Then I can apply the Woodbury identity of

$$
(\mathbf{A} + \mathbf{U} \mathbf{V}^\top)^{-1}=\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I} + \mathbf{V}^\top \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^\top \mathbf{A}^{-1},
$$
with $A$ being $\sigma^2 I,$ $U=\tilde Z$, and $V^\top = \tilde Z^\top$.

Then 
$$
\Omega^{-1}=\frac{1}{\sigma^2}I-\frac{1}{\sigma^4}\tilde Z(I+\frac{1}{\sigma^2}\tilde Z^\top \tilde Z)^{-1}\tilde Z^\top
$$
To finish this off, I'll Cholesky decompose $(I+\frac{1}{\sigma^2}\tilde Z^\top \tilde Z)=MM^\top$ to allow for easy inversion as $(MM^\top)^{-1}=M^{-\top}M^{-1}$. Then

$$
\begin{aligned}
\Omega^{-1}&=\frac{1}{\sigma^2}I-\frac{1}{\sigma^4}\tilde Z M^{-\top}M^{-1} \tilde Z^\top\\
&=\frac{1}{\sigma^2}I-\frac{1}{\sigma^4}(M^{-1} \tilde Z^\top)^\top(M^{-1} \tilde Z^\top).
\end{aligned}
$$

Sticking this into the quadratic form gives

$$
\begin{aligned}
-\frac{1}{2}(y_i-X_i\beta)^\top\Omega^{-1}(y_i-X_i\beta)&=-\frac{1}{2}(y_i-X_i\beta)^\top(\sigma^2\mathbf I+Z\Sigma Z^\top)^{-1}(y_i-X_i\beta)\\
&=-\frac{1}{2}(y_i-X_i\beta)^\top\big[\frac{1}{\sigma^2}I-\frac{1}{\sigma^4}(M^{-1} \tilde Z^\top)^\top(M^{-1}\tilde Z^\top)\big](y_i-X_i\beta)\\
&=-\frac{1}{2}(y_i-X_i\beta)^\top\big[\frac{1}{\sigma^2}I-\frac{1}{\sigma^4}A^\top A\big](y_i-X_i\beta), \text{ where } A:=M^{-1}\tilde Z^\top\\
&=-\frac{1}{2\sigma^2}(y_i-X_i\beta)^\top(y_i-X_i\beta)+\frac{1}{2\sigma^4}[A(y_i-X_i\beta)]^\top[A(y_i-X_i\beta)]
\end{aligned}
$$

The only other awkward term to compute is

$$
\begin{aligned}
\det(\Omega)&=\det(\sigma^2 I+Z\Sigma Z^\top)
\end{aligned}
$$

From the homework 1 identity, I have
$$
\begin{aligned}
\det(\sigma^2 I+\tilde Z\tilde Z^\top)=\det(\sigma^2 I)\det\Big(I+\frac{1}{\sigma^2}\tilde Z^\top\tilde Z\Big),
\end{aligned}
$$
Earlier I Cholesky decompose $I+\frac{1}{\sigma^2}\tilde Z^\top\tilde Z$ as $MM^\top$, so 
$$
\det(\sigma^2 I+\tilde Z\tilde Z^\top)=\det(\sigma^2 I)det(MM^\top)=(\sigma^2)^n\det(MM^\top)
$$

I can implement all of these terms to quickly compute in Julia to write the log-likelihood as
$$
\begin{align}
&-\frac{1}{2}\log|2\pi\Omega|-\frac{1}{2}(y_i-X_i\beta)^\top\Omega^{-1}(y_i-X_i\beta)\\
&=-\frac{n}{2}\log(2\pi)-\frac{n}{2}\log(\sigma^2)-\frac{1}{2}\log\det(MM^\top)-\frac{1}{2\sigma^2}(y_i-X_i\beta)^\top(y_i-X_i\beta)+\frac{1}{2\sigma^4}[A(y_i-X_i\beta)]^\top[A(y_i-X_i\beta)]
\end{align}
$$

In [319]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T}, 
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), size(obs.X, 2), size(obs.Z, 2) 
    # Compute y-Xβ
    mul!(obs.res, obs.X, β)    
    axpy!(-1, obs.y, obs.res)
    # Compute L'Z'ZL
    mul!(obs.storage_qq, L', obs.ztz)
    mul!(obs.storage_qq2, obs.storage_qq, L)
    # Compute I+1/σ²Z'Z
    mul!(obs.ztz, obs.storage_qq2, 1/σ²)
    for i=1:q
        obs.ztz[i, i] += 1
    end
    # Compute <A(y-Xβ),A(y-Xβ)> where A=M^{-1}Z'L'
    mul!(obs.storage_q, obs.Z', obs.res)
    mul!(obs.storage_q2, L', obs.storage_q)
    Ωchol = cholesky!(Symmetric(obs.ztz))
    # return full log-likelihood
    return -n/2 * log(2π) - n/2 * log(σ²) - 1/2 * logdet(Ωchol) - 
        1/(2 * σ²) * dot(obs.res, obs.res) + 
        1 / (2 * σ²^2) * dot(obs.storage_q2, Ωchol \  obs.storage_q2)
end

logl! (generic function with 1 method)

In [320]:
using BenchmarkTools, Distributions, LinearAlgebra, Random, SparseArrays, InteractiveUtils

Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([5.739048710854997, 5.705395720270055, 2.7368899643050355, 1.4201223592870755, -0.2099433929180451, 3.5886971824690486, -1.3778538474575956, -0.08406026821055246, -2.208007878450787, 1.309558511583542  …  1.2947876180172684, -1.9701265304395086, -2.040383092851745, -1.4590296825658675, 0.18616271231054726, 1.0681247149968018, 2.2292080864625254, 1.1952385354603545, 1.1310626949609706, -0.43507816286713785], [1.0 -2.506566300781151 … 0.5863780184080776 1.1092991040518192; 1.0 -0.974090320735282 … 1.4143507320583761 0.45608259198567447; … ; 1.0 -1.0076371084863895 … -1.3241972696483915 1.4547609424344008; 1.0 0.38036793320364776 … -0.5857507269707397 1.796804266836504], [1.0 -0.6380567326757537 1.4738982136806946; 1.0 -2.0711110232845926 0.21422658785510312; … ; 1.0 0.5917731507133951 -0.9163364468263059; 1.0 0.9463732120394507 -0.325860403600768], [NaN, -0.9431832160509631, 6.9386499169054e-310, -1.647480010986329, 6.93864991899354e-310, 0.3009450435638432, 6.93864991769

In [321]:
using Profile
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
@assert logl!(obs, β, Matrix(cholesky(Σ).L), σ²) ≈ logpdf(mvn, y)

In [322]:
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 
  memory estimate:  30.55 MiB
  allocs estimate:  5
  --------------
  minimum time:     9.633 ms (0.00% GC)
  median time:      14.108 ms (10.05% GC)
  mean time:        13.430 ms (16.29% GC)
  maximum time:     19.509 ms (23.32% GC)
  --------------
  samples:          372
  evals/sample:     1

In [323]:
# benchmark your implementation
L = Matrix(cholesky(Σ).L)
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 
  memory estimate:  224 bytes
  allocs estimate:  5
  --------------
  minimum time:     6.396 μs (0.00% GC)
  median time:      8.819 μs (0.00% GC)
  mean time:        9.122 μs (0.00% GC)
  maximum time:     30.184 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     4

In [324]:
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

30.0

In [325]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

29.78125